In [ ]:
"""
CloudOne Assistant - Minimal Multi-Agent Prototype

To run:
    python cloudone_assistant.py

This is a self-contained demo:
- Multi-agent orchestration
- GCP agent (stubbed)
- Workspace agent (stubbed)
- Policy agent
- Observability (logging)
- Simple session/memory
"""

from __future__ import annotations
import dataclasses
from dataclasses import dataclass, field
from typing import Dict, Any, List, Optional, Callable, Tuple
import datetime
import uuid
import re
import logging

# ---------------------------------------------------------------------
# Logging / Observability setup
# ---------------------------------------------------------------------

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)


def new_request_id() -> str:
    return str(uuid.uuid4())[:8]


# ---------------------------------------------------------------------
# Very simple "LLM" stub (for now)
# ---------------------------------------------------------------------

def call_llm(system_prompt: str, conversation: List[Dict[str, str]]) -> str:
    """
    Stub LLM call.

    For a real system, replace this with:
    - Gemini API
    - OpenAI API
    - Vertex AI, etc.

    Here we just do a tiny rule-based reply to keep the demo executable.
    """
    last_user = ""
    for m in reversed(conversation):
        if m["role"] == "user":
            last_user = m["content"]
            break

    # Tiny pattern for acknowledgement
    if "create" in last_user.lower() and "vm" in last_user.lower():
        return "I will ask the GCP Agent to create a VM with your requested parameters."
    if "project" in last_user.lower():
        return "I will ask the GCP Agent to manage the project you mentioned."
    if "schedule" in last_user.lower() or "meeting" in last_user.lower():
        return "I will ask the Workspace Agent to schedule a calendar event and Meet link."
    return "Got it. I will coordinate the agents to handle your request."


# ---------------------------------------------------------------------
# Data structures
# ---------------------------------------------------------------------

@dataclass
class ToolResult:
    success: bool
    data: Any = None
    error: Optional[str] = None


@dataclass
class SessionState:
    user_id: str
    current_project: Optional[str] = None
    default_region: str = "us-central1"
    preferences: Dict[str, Any] = field(default_factory=dict)

    def summary(self) -> str:
        return f"SessionState(user={self.user_id}, project={self.current_project}, region={self.default_region})"


# ---------------------------------------------------------------------
# Base Agent
# ---------------------------------------------------------------------

class BaseAgent:
    name: str

    def __init__(self, observability: "ObservabilityAgent"):
        self.observability = observability

    def log(self, request_id: str, message: str):
        self.observability.log(self.name, request_id, message)


# ---------------------------------------------------------------------
# Observability Agent
# ---------------------------------------------------------------------

class ObservabilityAgent:
    def __init__(self):
        self.logs: List[Dict[str, Any]] = []

    def log(self, agent_name: str, request_id: str, message: str):
        record = {
            "timestamp": datetime.datetime.utcnow().isoformat(),
            "agent": agent_name,
            "request_id": request_id,
            "message": message,
        }
        self.logs.append(record)
        logging.info(f"[{request_id}] [{agent_name}] {message}")

    def get_logs_for_request(self, request_id: str) -> List[Dict[str, Any]]:
        return [l for l in self.logs if l["request_id"] == request_id]


# ---------------------------------------------------------------------
# Policy / Guardrail Agent
# ---------------------------------------------------------------------

class PolicyAgent(BaseAgent):
    def __init__(self, observability: ObservabilityAgent):
        super().__init__(observability)
        self.name = "PolicyAgent"

        # Example policies
        self.allowed_regions = {"us-central1", "europe-west1"}
        self.max_machine_type = "n2-standard-8"  # just a label for demo

    def validate_vm_request(
        self,
        request_id: str,
        project_id: str,
        region: str,
        machine_type: str,
        labels: Dict[str, str],
    ) -> ToolResult:
        self.log(request_id, f"Validating VM request for project={project_id}, region={region}, machine_type={machine_type}, labels={labels}")

        if region not in self.allowed_regions:
            return ToolResult(
                success=False,
                error=f"Region {region} is not allowed by policy. Allowed: {sorted(self.allowed_regions)}",
            )

        # Simple label policy
        required_labels = ["env", "owner"]
        missing = [k for k in required_labels if k not in labels]
        if missing:
            return ToolResult(
                success=False,
                error=f"Missing required labels: {missing}. Please include them.",
            )

        # For demo, we don't implement numeric machine-type size check.
        # In a real system, parse machine type and compare.

        return ToolResult(success=True, data="VM request validated")


# ---------------------------------------------------------------------
# GCP Resource Agent (stubbed)
# ---------------------------------------------------------------------

class GCPAgent(BaseAgent):
    def __init__(self, observability: ObservabilityAgent):
        super().__init__(observability)
        self.name = "GCPAgent"

        # In-memory "fake" store
        self.projects: Dict[str, Dict[str, Any]] = {}
        self.vms: Dict[str, Dict[str, Any]] = {}

    # ------------------ Project methods ------------------

    def create_project(self, request_id: str, project_id: str, labels: Dict[str, str]) -> ToolResult:
        self.log(request_id, f"create_project called with project_id={project_id}, labels={labels}")

        if project_id in self.projects:
            return ToolResult(success=False, error=f"Project {project_id} already exists")

        # TODO: Replace this with real GCP API call
        self.projects[project_id] = {
            "project_id": project_id,
            "labels": labels,
            "created_at": datetime.datetime.utcnow().isoformat(),
        }

        return ToolResult(success=True, data=self.projects[project_id])

    def list_projects(self, request_id: str) -> ToolResult:
        self.log(request_id, "list_projects called")
        # TODO: Replace with GCP resource manager API if desired
        return ToolResult(success=True, data=list(self.projects.values()))

    # ------------------ VM methods ------------------

    def create_vm(
        self,
        request_id: str,
        name: str,
        project_id: str,
        region: str,
        machine_type: str,
        ttl_hours: float,
        labels: Dict[str, str],
    ) -> ToolResult:
        self.log(request_id, f"create_vm called with name={name}, project={project_id}, region={region}, machine_type={machine_type}, ttl={ttl_hours}, labels={labels}")

        if project_id not in self.projects:
            return ToolResult(success=False, error=f"Project {project_id} does not exist")

        vm_id = f"{project_id}/{name}"
        if vm_id in self.vms:
            return ToolResult(success=False, error=f"VM {vm_id} already exists")

        expires_at = datetime.datetime.utcnow() + datetime.timedelta(hours=ttl_hours)

        # TODO: Replace with real Compute Engine API
        self.vms[vm_id] = {
            "vm_id": vm_id,
            "name": name,
            "project_id": project_id,
            "region": region,
            "machine_type": machine_type,
            "labels": labels,
            "status": "RUNNING",
            "created_at": datetime.datetime.utcnow().isoformat(),
            "expires_at": expires_at.isoformat(),
        }

        return ToolResult(success=True, data=self.vms[vm_id])

    def list_vms(self, request_id: str, project_id: Optional[str] = None) -> ToolResult:
        self.log(request_id, f"list_vms called for project={project_id}")
        if project_id:
            vms = [v for v in self.vms.values() if v["project_id"] == project_id]
        else:
            vms = list(self.vms.values())
        return ToolResult(success=True, data=vms)

    def update_vm_status(self, request_id: str, project_id: str, name: str, new_status: str) -> ToolResult:
        vm_id = f"{project_id}/{name}"
        self.log(request_id, f"update_vm_status {vm_id} -> {new_status}")
        vm = self.vms.get(vm_id)
        if not vm:
            return ToolResult(success=False, error=f"VM {vm_id} not found")
        vm["status"] = new_status
        return ToolResult(success=True, data=vm)

    def stop_vm(self, request_id: str, project_id: str, name: str) -> ToolResult:
        # TODO: Replace with real Compute Engine stop call
        return self.update_vm_status(request_id, project_id, name, "STOPPED")

    def start_vm(self, request_id: str, project_id: str, name: str) -> ToolResult:
        # TODO: Replace with real Compute Engine start call
        return self.update_vm_status(request_id, project_id, name, "RUNNING")


# ---------------------------------------------------------------------
# Workspace Agent (Calendar + Meet, stubbed)
# ---------------------------------------------------------------------

class WorkspaceAgent(BaseAgent):
    def __init__(self, observability: ObservabilityAgent):
        super().__init__(observability)
        self.name = "WorkspaceAgent"
        self.events: Dict[str, Dict[str, Any]] = {}

    def create_event(
        self,
        request_id: str,
        title: str,
        start_time: datetime.datetime,
        duration_minutes: int,
        attendees: List[str],
        description: str,
        project_id: Optional[str] = None,
    ) -> ToolResult:
        self.log(request_id, f"create_event title={title}, start={start_time}, duration={duration_minutes}, attendees={attendees}, project={project_id}")

        event_id = str(uuid.uuid4())
        end_time = start_time + datetime.timedelta(minutes=duration_minutes)

        # TODO: Replace this with Google Calendar API call (with conferenceData to create Meet link)
        meet_link = f"https://meet.google.com/{event_id[:3]}-{event_id[3:6]}-{event_id[6:9]}"

        event = {
            "event_id": event_id,
            "title": title,
            "start_time": start_time.isoformat(),
            "end_time": end_time.isoformat(),
            "attendees": attendees,
            "description": description,
            "project_id": project_id,
            "meet_link": meet_link,
        }
        self.events[event_id] = event

        return ToolResult(success=True, data=event)


# ---------------------------------------------------------------------
# Orchestrator Agent
# ---------------------------------------------------------------------

class OrchestratorAgent(BaseAgent):
    def __init__(
        self,
        observability: ObservabilityAgent,
        gcp_agent: GCPAgent,
        workspace_agent: WorkspaceAgent,
        policy_agent: PolicyAgent,
        session_state: SessionState,
    ):
        super().__init__(observability)
        self.name = "OrchestratorAgent"
        self.gcp = gcp_agent
        self.workspace = workspace_agent
        self.policy = policy_agent
        self.session = session_state
        self.history: List[Dict[str, str]] = []

    # --------------- High-level entry point ---------------

    def handle_user_message(self, request_id: str, user_message: str) -> str:
        self.log(request_id, f"User message: {user_message}")
        self.history.append({"role": "user", "content": user_message})

        # Recognize some "context" commands first
        context_reply = self._maybe_handle_context_command(request_id, user_message)
        if context_reply:
            self.history.append({"role": "assistant", "content": context_reply})
            return context_reply

        # Lightweight routing based on keywords for this demo.
        # In a real system, you'd use the LLM to decide which tools/agents to invoke.
        response_chunks: List[str] = []

        lower = user_message.lower()

        try:
            if "create" in lower and "vm" in lower:
                resp = self._handle_create_vm(request_id, user_message)
                response_chunks.append(resp)
            elif "list" in lower and "vm" in lower:
                resp = self._handle_list_vms(request_id, user_message)
                response_chunks.append(resp)
            elif "create" in lower and "project" in lower:
                resp = self._handle_create_project(request_id, user_message)
                response_chunks.append(resp)
            elif "list" in lower and "project" in lower:
                resp = self._handle_list_projects(request_id)
                response_chunks.append(resp)
            elif "schedule" in lower or "meeting" in lower or "calendar" in lower:
                resp = self._handle_schedule_event(request_id, user_message)
                response_chunks.append(resp)
            else:
                # Fallback: still call LLM stub for a friendly response
                system_prompt = "You are the Orchestrator Agent for CloudOne Assistant."
                llm_reply = call_llm(system_prompt, self.history)
                response_chunks.append(llm_reply)
        except Exception as e:
            self.log(request_id, f"Error in orchestrator: {e}")
            response_chunks.append(f"An error occurred while handling your request: {e}")

        full_response = "\n".join(response_chunks)
        self.history.append({"role": "assistant", "content": full_response})
        return full_response

    # --------------- Context / session commands ---------------

    def _maybe_handle_context_command(self, request_id: str, msg: str) -> Optional[str]:
        # Example: "set project to my-project-123"
        m = re.search(r"set\s+project\s+to\s+([a-zA-Z0-9\-]+)", msg, re.IGNORECASE)
        if m:
            project_id = m.group(1)
            self.session.current_project = project_id
            self.log(request_id, f"Session project set to {project_id}")
            return f"Okay, I will use **{project_id}** as the current project."

        # Example: "set default region to us-central1"
        m = re.search(r"set\s+default\s+region\s+to\s+([a-z0-9\-]+)", msg, re.IGNORECASE)
        if m:
            region = m.group(1)
            self.session.default_region = region
            self.log(request_id, f"Session default_region set to {region}")
            return f"Okay, I will use **{region}** as the default region."

        return None

    # --------------- Helpers to parse simple inputs ---------------

    def _extract_project(self, msg: str) -> Optional[str]:
        # Try "project X"
        m = re.search(r"project\s+([a-zA-Z0-9\-]+)", msg, re.IGNORECASE)
        if m:
            return m.group(1)
        return self.session.current_project

    def _extract_region(self, msg: str) -> str:
        m = re.search(r"(us-[a-z0-9\-]+|europe-[a-z0-9\-]+)", msg, re.IGNORECASE)
        if m:
            return m.group(1)
        return self.session.default_region

    def _extract_machine_type(self, msg: str) -> str:
        m = re.search(r"(n2-[a-z0-9\-]+|e2-[a-z0-9\-]+)", msg, re.IGNORECASE)
        if m:
            return m.group(1)
        return "n2-standard-4"

    def _extract_ttl_hours(self, msg: str) -> float:
        m = re.search(r"(\d+)\s*(hour|hours|hr|hrs)", msg, re.IGNORECASE)
        if m:
            return float(m.group(1))
        return 4.0

    def _extract_vm_name(self, msg: str) -> str:
        m = re.search(r"vm\s+named\s+([a-zA-Z0-9\-]+)", msg, re.IGNORECASE)
        if m:
            return m.group(1)
        return f"vm-{uuid.uuid4().hex[:6]}"

    def _extract_labels(self, msg: str) -> Dict[str, str]:
        # Very simple: "env=dev owner=alice"
        labels = {}
        for match in re.findall(r"([a-zA-Z_]+)=([a-zA-Z0-9_\-]+)", msg):
            key, value = match
            labels[key] = value
        # Provide some default if missing
        if "env" not in labels:
            labels["env"] = "dev"
        if "owner" not in labels:
            labels["owner"] = self.session.user_id
        return labels

    # --------------- Handlers ---------------

    def _handle_create_project(self, request_id: str, msg: str) -> str:
        project_id = self._extract_project(msg)
        if not project_id:
            return "I could not determine the project ID. Please specify it like: `create project my-project-123`."

        labels = self._extract_labels(msg)
        res = self.gcp.create_project(request_id, project_id, labels)
        if not res.success:
            return f"Failed to create project `{project_id}`: {res.error}"
        self.session.current_project = project_id
        return f"Created project `{project_id}` with labels {labels}. I’ve also set it as the current project."

    def _handle_list_projects(self, request_id: str) -> str:
        res = self.gcp.list_projects(request_id)
        if not res.success:
            return f"Failed to list projects: {res.error}"
        if not res.data:
            return "There are no projects yet."
        lines = ["📂 Projects:"]
        for p in res.data:
            lines.append(f"- `{p['project_id']}` labels={p['labels']}")
        return "\n".join(lines)

    def _handle_create_vm(self, request_id: str, msg: str) -> str:
        project_id = self._extract_project(msg)
        if not project_id:
            return "I could not determine the project. Please either set a current project or say `in project X`."

        region = self._extract_region(msg)
        machine_type = self._extract_machine_type(msg)
        ttl_hours = self._extract_ttl_hours(msg)
        vm_name = self._extract_vm_name(msg)
        labels = self._extract_labels(msg)

        # Policy check
        policy_result = self.policy.validate_vm_request(request_id, project_id, region, machine_type, labels)
        if not policy_result.success:
            return f"Policy blocked VM creation: {policy_result.error}"

        res = self.gcp.create_vm(
            request_id,
            name=vm_name,
            project_id=project_id,
            region=region,
            machine_type=machine_type,
            ttl_hours=ttl_hours,
            labels=labels,
        )
        if not res.success:
            return f"Failed to create VM: {res.error}"

        vm = res.data
        return (
            "VM created:\n"
            f"- Name: `{vm['name']}`\n"
            f"- Project: `{vm['project_id']}`\n"
            f"- Region: `{vm['region']}`\n"
            f"- Machine type: `{vm['machine_type']}`\n"
            f"- Status: `{vm['status']}`\n"
            f"- Expires at (UTC): `{vm['expires_at']}`"
        )

    def _handle_list_vms(self, request_id: str, msg: str) -> str:
        project_id = self._extract_project(msg)
        res = self.gcp.list_vms(request_id, project_id=project_id)
        if not res.success:
            return f"Failed to list VMs: {res.error}"
        vms = res.data
        if not vms:
            return "No VMs found for that project." if project_id else "No VMs found."
        lines = ["VMs:"]
        for v in vms:
            lines.append(
                f"- `{v['vm_id']}` status={v['status']} region={v['region']} labels={v['labels']}"
            )
        return "\n".join(lines)

    def _handle_schedule_event(self, request_id: str, msg: str) -> str:
        # Very basic parser: detect time like "at 3pm" or "at 15:00"
        # For the demo, we just schedule at "today HH:00" or "now + 1h" fallback.

        title = "Cloud maintenance / meeting"
        project_id = self._extract_project(msg) or self.session.current_project

        # Duration: look for "for 30 minutes"
        m = re.search(r"for\s+(\d+)\s*(min|mins|minutes)", msg, re.IGNORECASE)
        duration = int(m.group(1)) if m else 30

        # Time: naive parse "at 3pm" / "at 15:00"
        start_time = datetime.datetime.utcnow() + datetime.timedelta(hours=1)
        time_match_12 = re.search(r"at\s+(\d{1,2})\s*(am|pm)", msg, re.IGNORECASE)
        time_match_24 = re.search(r"at\s+(\d{1,2}):(\d{2})", msg, re.IGNORECASE)

        if time_match_12:
            hour = int(time_match_12.group(1))
            ampm = time_match_12.group(2).lower()
            if ampm == "pm" and hour != 12:
                hour += 12
            elif ampm == "am" and hour == 12:
                hour = 0
            start_time = datetime.datetime.utcnow().replace(
                hour=hour, minute=0, second=0, microsecond=0
            )
        elif time_match_24:
            hour = int(time_match_24.group(1))
            minute = int(time_match_24.group(2))
            start_time = datetime.datetime.utcnow().replace(
                hour=hour, minute=minute, second=0, microsecond=0
            )

        # Attendees: look for "with alice@example.com, bob@example.com"
        attendees_match = re.search(r"with\s+(.+)", msg, re.IGNORECASE)
        attendees: List[str] = []
        if attendees_match:
            raw = attendees_match.group(1)
            attendees = [a.strip() for a in re.split(r"[,\s]+", raw) if "@" in a]

        description_parts = [
            "Scheduled via CloudOne Assistant.",
        ]
        if project_id:
            description_parts.append(f"Project: {project_id}")
        description = " ".join(description_parts)

        res = self.workspace.create_event(
            request_id=request_id,
            title=title,
            start_time=start_time,
            duration_minutes=duration,
            attendees=attendees,
            description=description,
            project_id=project_id,
        )
        if not res.success:
            return f"Failed to schedule event: {res.error}"

        ev = res.data
        return (
            "📅 Meeting scheduled:\n"
            f"- Title: {ev['title']}\n"
            f"- Start (UTC): {ev['start_time']}\n"
            f"- End (UTC): {ev['end_time']}\n"
            f"- Meet link: {ev['meet_link']}\n"
            f"- Attendees: {', '.join(ev['attendees']) if ev['attendees'] else 'none'}\n"
            f"- Project: {ev['project_id'] or 'n/a'}"
        )


# ---------------------------------------------------------------------
# Main REPL
# ---------------------------------------------------------------------

def main():
    print("=== CloudOne Assistant (Demo) ===")
    print("Type 'exit' to quit.")
    print("Examples:")
    print("  - set project to team-dev")
    print("  - create project team-dev env=dev owner=alice")
    print("  - create a vm in us-central1 with n2-standard-4 for 4 hours env=dev owner=alice")
    print("  - list vms in project team-dev")
    print("  - schedule a meeting at 15:00 for 30 minutes with alice@example.com bob@example.com in project team-dev")
    print()

    user_id = "demo-user"
    observability = ObservabilityAgent()
    session_state = SessionState(user_id=user_id)
    gcp_agent = GCPAgent(observability)
    workspace_agent = WorkspaceAgent(observability)
    policy_agent = PolicyAgent(observability)
    orchestrator = OrchestratorAgent(
        observability=observability,
        gcp_agent=gcp_agent,
        workspace_agent=workspace_agent,
        policy_agent=policy_agent,
        session_state=session_state,
    )

    while True:
        try:
            user_msg = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nGoodbye.")
            break

        if user_msg.lower() in {"exit", "quit"}:
            print("Goodbye.")
            break

        request_id = new_request_id()
        reply = orchestrator.handle_user_message(request_id, user_msg)
        print(f"\nCloudOne: {reply}\n")


if __name__ == "__main__":
    main()
